## QC of Ingestion (not ready to use!!)

This notebook checks the original RASTER data information against EDC data.

A list of QC-tests is carried out for each tile and a QC-file is created:
-> [QC_environmental_zones_1km.txt]






In [27]:
# Configure plots for inline use in Jupyter Notebook
%matplotlib inline
import datetime as dt
from datetime import datetime
# Utilities
import boto3
import dateutil
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os
import rasterio
import rasterio.mask
from rasterio.plot import show_hist
from rasterio.plot import show
from rasterio.windows import Window
from rasterio.windows import from_bounds # for aoi

from pyreproj import Reprojector

import random
import fiona
import numpy as np
from shapely.geometry import mapping, Polygon
from shapely import geometry
import logging    

# Sentinel Hub
from sentinelhub import (
    CRS,
    BBox,
    ByocCollection,
    ByocCollectionAdditionalData,
    ByocCollectionBand,
    ByocTile,
    DataCollection,
    DownloadFailedException,
    MimeType,
    SentinelHubBYOC,
    SentinelHubRequest,
    SHConfig,
    bbox_to_dimensions,
    os_utils,
)

config = SHConfig()
config.instance_id = os.environ.get("SH_INSTANCE_ID")
config.sh_client_id = os.environ.get("SH_CLIENT_ID")
config.sh_client_secret = os.environ.get("SH_CLIENT_SECRET")
config.aws_access_key_id = os.environ.get("username")
config.aws_secret_access_key = os.environ.get("password")
    
print ("reading of libaries done!!")
    

reading of libaries done!!


Please set up the correct links to input raster dataset and the ingested collection:

In [35]:
# Read raster files on S3 drive
#### CHECKING ENV ZONES:

##########################SET name and data folder: ################################################################################ S
name_of_ingestion = "environmental_zones_1km"               # should be the collection name!!!"
collection_id      ='5b45916e-6704-4581-824f-4d713198731b'  # collection ID 
input_raster ="./../../../../s3/data/d005_env_zones/raw_env_zones/env_zones_1km_3035.tif"   ## conect to tile folder on S3



##################################################################################################################################### E



########################## BBOX ##################################################################################################### S
## SET UP BBOX for QC
#https://rasterio.readthedocs.io/en/stable/api/rasterio.windows.html#rasterio.windows.from_bounds
x1 =     4014674  # left
y1=      2933830  # Top
x2 =x1 + 2000     # Right
y2 =y1 - 2000     # Bottom
resolution = 100
bbox_coords = x1, y1, x2, y2
########################## BBOX ###################################################################################################### E

print ("reading of input paramters done!!")

reading of input paramters done!!


In [26]:
from datetime import datetime

current_dateTime = datetime.now()

print(current_dateTime)

2023-12-14 12:03:03.007117


In [86]:
# START
# assign s3 directory
#directory = input_folder
#print (directory)
current_dateTime = datetime.now()

print ("following raster is selected for QC:")
##################################### QC log file set up:#################################### S
#now we will Create and configure logger 
#https://www.geeksforgeeks.org/reading-writing-text-files-python/
qc_log = open("QC_"+name_of_ingestion+"_raster_edc.txt","w")
qc_log.write("QC_" +name_of_ingestion+" \n")
qc_log.write("---------------------------------------------------------- \n")
qc_log.write("START-time: " +str(current_dateTime)+" \n")
############################################################################################# E



###############################################################open raster: inside bbox ############ S
raster_for_qc = input_raster
raster = rasterio.open(raster_for_qc)
qc_log.write("(1) CHECK 1 - spatial check for raster files:   \n")
qc_log.write("----------------------------------------------   \n")
# read different raster information from s3 raster file (tiles):
raster_name = raster.name
raster_bands = raster.count
raster_width= raster.width
raster_height= raster.height
raster_bounds= raster.bounds

qc_log.write( raster_name + "   \n ")
#
left = raster_bounds[0]
bottom = raster_bounds[1]
right = raster_bounds[2]
top = raster_bounds[3]
raster_extent= raster.bounds

raster_transform =raster.transform
pixelSizeX = raster_transform[0]
pixelSizeY =-raster_transform[4]
raster_crs =raster.crs
raster_dtype = raster.dtypes[0]
raster_nodata = raster.nodata
                
with rasterio.open(raster_for_qc) as src:
    #rst = src.read(1, window=from_bounds(left, bottom, right, top, src.transform))
    band_data = src.read(1, window=from_bounds(x1, y2, x2, y1, src.transform))
    r_min_raster = band_data.min() 
    r_max_raster = band_data.max() 
    r_mean_raster = band_data.mean() 
    
print (r_min_raster)
print (r_max_raster)
print (r_mean_raster)
 
    
    

###############################################################open CUBE collection: ############ S
# Initialize SentinelHubBYOC class
byoc = SentinelHubBYOC(config=config)
#############################################################
name_of_your_collection = name_of_ingestion
collections_iterator = byoc.iter_collections(name_of_your_collection)
my_collection = list(collections_iterator)[0]
#tiles = list(byoc.iter_tiles(my_collection))
############################################################################################# E
#print (my_collection)
print (bbox_coords)
lux_bbox=  BBox(bbox=bbox_coords, crs=CRS('3035').pyproj_crs())
lux_size = bbox_to_dimensions(lux_bbox, resolution=resolution)
print(f"Image shape at {resolution} m resolution: {lux_size} pixels")
data_collection_aoi = DataCollection.define_byoc(collection_id, name=name_of_ingestion)

# simple evalscript to read the for bounding box inside the cube-tile:
evalscript_test = """

//VERSION=3
function setup() {
  return {
    input: ["B01"],
    output: { 
        bands: 1,
        sampleType: "UINT16" // raster format will be UINT16
        }

  };
}

function evaluatePixel(sample) {
  return [sample.B01];
}
"""

#test = 'DataCollection.'+name_of_ingestion  #DataCollection.environmental_zones_1km

print(data_collection_aoi)
request = SentinelHubRequest(
        evalscript=evalscript_test,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=data_collection_aoi,
                             
            )
        ],
        responses=[
            SentinelHubRequest.output_response('default', MimeType.PNG)
        ],
        bbox=lux_bbox,
        size=bbox_to_dimensions(lux_bbox, 100),
        config=config)

data = request.get_data()[0]

data_min_cube = np.min(data)
data_max_cube = np.max(data)
data_mean_cube= np.mean(data)

print (data_min_cube)
print (data_max_cube)
print (data_mean_cube)

############################################################


bitDepth = my_collection['additionalData']['bands']['B01']['bitDepth']
number_of_bands = my_collection['additionalData']['bands']['B01']['bandIndex']
data_type = my_collection['additionalData']['bands']['B01']['sampleFormat']
#print (my_collection)

tiles = list(byoc.iter_tiles(my_collection))  # read first tile from collection to get EPSG code information:
tile =tiles[0]



EPSG_code = tile['coverGeometry'][ 'crs']['properties'][ 'name'].split(':')[-1]
#qc_log.write ("ESPG: "+str(EPSG_code)           + " \n")
#print ("---------")
cube_epsg_str =  ("EPSG:"+str(EPSG_code))
#print (cube_epsg_str)

pixelSize_cube_min = tile['additionalData']['minMetersPerPixel'] 

######### Get Extent by checking all single tiles: #################################S
top_left_x_list 	= []
top_left_y_list     = []
top_right_x_list    = []
top_right_y_list    = []
bottom_left_x_list  = []
bottom_left_y_list  = []
bottom_right_x_list = []
bottom_right_y_list = []


for tile in tiles:
    tile_extent=  tile['tileGeometry'] ['coordinates']
    

    top_left_x_list.append(tile_extent[0][0][0] )
    top_left_y_list.append(tile_extent[0][0][1]  )
    top_right_x_list.append(tile_extent[0][1][0]   )
    top_right_y_list.append(tile_extent[0][1][1]  )
    bottom_left_x_list.append( tile_extent[0][2][0]  )
    bottom_left_y_list.append( tile_extent[0][2][1]  )
    bottom_right_x_list.append(tile_extent[0][3][0]  )
    bottom_right_y_list.append(tile_extent[0][3][1]  )
    
left_cube=  (min(top_left_x_list))  ## left
top_cube=  (max(top_left_y_list))  ## top
right_cube=  (max(top_right_x_list))  ## right
bottom_cube=  (min(bottom_right_y_list))  ## bottom



######### Get Extent by checking all single tiles: #################################E


###############################################################open CUBE collection: ############ E



############################################################### TESTS:              ############ S

 
qc_log.write ("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\n")  
print ("check 1.1:.....") ########################################################################################check 1.1 CRS
EPSG_code = tile['coverGeometry'][ 'crs']['properties'][ 'name'].split(':')[-1]
#qc_log.write ("ESPG: "+str(EPSG_code)           + " \n")
#print ("---------")
cube_epsg_str =  ("EPSG:"+str(EPSG_code))

qc_log.write ("check 1.1 (CRS) -START \n")   
qc_log.write ("-RASTER:  \n")
qc_log.write ("  EPSG code: " +str(raster_crs) +' \n')    
qc_log.write ("-CUBE_TILE:  \n")
qc_log.write ("  EPSG code: " +str(cube_epsg_str) +' \n')            
raster_epsg_str =  (str(raster_crs))
#print ("---------")

if cube_epsg_str == raster_epsg_str:
    qc_log.write ("check 1.1 - EPSG (crs): OK   \n")
else:
    qc_log.write ("check 1.1 - EPSG (crs): NOT-OK  \n")

qc_log.write ("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\n")   
print ("check 1.2:.....") ########################################################################################check 1.2 cell size
qc_log.write ("check 1.2 (cellsize)-START  \n")  
qc_log.write ("-RASTER:  \n")
qc_log.write ("  Pixel size x: " +str(pixelSizeX) +' \n')
qc_log.write ("  Pixel size y: " +str(pixelSizeY) +' \n')

qc_log.write ("-CUBE_TILE:  \n")
qc_log.write ("  CUBE size x: " +str(pixelSize_cube_min) +' \n')
qc_log.write ("  CUBE size y: " +str(pixelSize_cube_min) +' \n')

if str(pixelSizeX) == str(pixelSize_cube_min)  and str(pixelSizeY) == str(pixelSize_cube_min) :                     
    qc_log.write ("check 1.2 - cell size  :  OK   \n")
else:
    qc_log.write ("check 1.2 -  cell size :  NOT-OK  \n")

    
qc_log.write ("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\n")  
print ("check 1.3:.....") ########################################################################################check 1.3 tile size extend:
qc_log.write ("check 1.3 (extend)-START \n")  
qc_log.write ("-RASTER:  \n")
qc_log.write ("  extent: "+str(raster_extent)     + " \n")
qc_log.write ("  with: "+str(raster_width)     + " \n")
qc_log.write ("  height: "+str(raster_height)    + " \n")
qc_log.write ("  --------  \n")
qc_log.write ("  left: "+str(left)+'        \n')
qc_log.write ("  bottom: " +str(bottom)+'    \n')
qc_log.write ("  right: "+str(right)+'       \n')
qc_log.write ("  top: " +str(top)+'            \n')

qc_log.write ("-CUBE_TILE:  \n")
qc_log.write ("  left: "   +str(left_cube)+'         \n')
qc_log.write ("  bottom: " +str(bottom_cube)+'         \n')
qc_log.write ("  right: "  +str(right_cube)  +'         \n')
qc_log.write ("  top: "    +str(top_cube)  +'         \n')


if  str(left) == str(left_cube)  and str(bottom) == str(bottom_cube)  and str(right) == str(right_cube)  and str(top) == str(top_cube)  :                  
    qc_log.write ("check 1.3 - extend  :  OK   \n")
else:
    qc_log.write ("check 1.3 - extend :  NOT-OK  \n")
    
    
    


#pixelSizeX = raster_transform[0]   ## raster
#pixelSizeY =-raster_transform[4]    ###raster
qc_log.write ("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\n")  
print ("check 1.4:.....")             
qc_log.write ("check 1.4 ( statistics)-START  \n")  
#https://sentinelhub-py.readthedocs.io/en/latest/examples/byoc_request.html
##print ("NEXT file:.....")

qc_log.write ("-RASTER:  \n")
#qc_log.write ("  raster data type: "    + str(raster_dtype)     + " \n")
#qc_log.write ("  raster nodata value: " + str(raster_nodata)    + " \n")  
qc_log.write ("  max raster value: "  +str(r_max_raster) +' \n')
qc_log.write ("  min raster value: " +str(r_min_raster) +' \n')
qc_log.write ("  avg raster value: " +str(r_mean_raster) +' \n')
qc_log.write ("-CUBE_TILE:  \n")
qc_log.write ("  max cube value: "  +str(data_max_cube) +' \n')
qc_log.write ("  min cube value: " +str(data_min_cube) +' \n')
qc_log.write ("  avg cube  value: " +str(data_mean_cube) +' \n')


if str(r_max_raster) == str(data_max_cube)  and str(r_min_raster) == str(data_min_cube) :                     
    qc_log.write ("check 1.4 - data : OK   \n")
else:
    qc_log.write ("check 1.4 - data :  NOT-OK  \n")


qc_log.write ("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\n")  
print ("check 1.5:.....")  
qc_log.write ("check 1.5 (data type) -START \n")  

qc_log.write ("-RASTER:  \n")
qc_log.write ("  raster data type: "    +str(raster_dtype)     + " \n")
qc_log.write ("  raster nodata value: " + str(raster_nodata)    + " \n")  

qc_log.write ("-CUBE_TILE:  \n")
qc_log.write ("  cube data type: "    +   data_type.lower()+str(bitDepth)     + " \n")
qc_log.write ("  cube nodata value: "    + " \n")  


if data_type.lower()+str(bitDepth) == str(raster_dtype)   :                     
    qc_log.write ("check 1.5 - data : OK   \n")
else:
    qc_log.write ("check 1.5 - data :  NOT-OK  \n")


qc_log.write ("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\n")  
print ("check 1.6:.....")  
qc_log.write ("check 1.6 (number of bands) -START \n")  

qc_log.write ("-RASTER:  \n")
qc_log.write ("  raster number of bands: "    + str(raster_bands)     + " \n")
  

qc_log.write ("-CUBE_TILE:  \n")
qc_log.write ("  cube number of bands: "    + str(number_of_bands)     + " \n")

if str(raster_bands) == str(number_of_bands) :                     
    qc_log.write ("check 1.6 - #bands : OK   \n")
else:
    qc_log.write ("check 1.6 - #bands :  NOT-OK  \n")

############################################################### TESTS:              ############ E







#######################################################################################################################################
qc_log.write("------------------------------------------------------------------------------------ \n")
current_dateTime = datetime.now()
qc_log.write("END-time: " +str(current_dateTime)+" \n")
print ("end")


following raster is selected for QC:
7
7
7.0
(4014674, 2933830, 4016674, 2931830)
Image shape at 100 m resolution: (18, 22) pixels
DataCollection.environmental_zones_1km
7
7
7.0
check 1.1:.....
check 1.2:.....
check 1.3:.....
check 1.4:.....
check 1.5:.....
check 1.6:.....
end


In [37]:
## TESTING

print (my_collection['additionalData']['bands']['B01'])
print ('bitDepth: ' + str(my_collection['additionalData']['bands']['B01']['bitDepth']))
print (my_collection['additionalData']['bands']['B01'])
print (my_collection)
print ("-------------------")
tiles = list(byoc.iter_tiles(my_collection))
print ("-------------------")
print (my_collection['additionalData']['extent']['coordinates'])
print ("-------------------")

bitDepth = my_collection['additionalData']['bands']['B01']['bitDepth']
number_of_bands = my_collection['additionalData']['bands']['B01']['bandIndex']
data_type = my_collection['additionalData']['bands']['B01']['sampleFormat']

print(tiles[0])
EPSG_code = tile['coverGeometry'][ 'crs']['properties'][ 'name'].split(':')[-1]
#qc_log.write ("ESPG: "+str(EPSG_code)           + " \n")
#print ("---------")
cube_epsg_str =  ("EPSG:"+str(EPSG_code))
print (cube_epsg_str)

print ("-------------------")
print ( tile['additionalData'][ 'minMetersPerPixel']   )


    


{'bitDepth': 8, 'source': 'B01', 'bandIndex': 1, 'sampleFormat': 'UINT'}
bitDepth: 8
{'bitDepth': 8, 'source': 'B01', 'bandIndex': 1, 'sampleFormat': 'UINT'}
{'id': '5b45916e-6704-4581-824f-4d713198731b', 'accountId': 'd4403346-93d0-4ea6-989b-d7a990989ca6', 'name': 'environmental_zones_1km', 's3Bucket': 'hub-fairicube0', 'additionalData': {'bands': {'B01': {'bitDepth': 8, 'source': 'B01', 'bandIndex': 1, 'sampleFormat': 'UINT'}}, 'maxMetersPerPixel': 8000.0, 'extent': {'type': 'Polygon', 'coordinates': [[[-56.505141902, 24.284177011], [-56.505141902, 72.663979307], [72.906136759, 72.663979307], [72.906136759, 24.284177011], [-56.505141902, 24.284177011]]]}, 'fromSensingTime': '2018-01-01T00:00:00Z', 'toSensingTime': '2018-01-01T00:00:00Z', 'hasSensingTimes': 'YES'}, 'noData': 0, 'created': '2023-12-12T14:21:31.638504Z', 'requiresMetadataUpdate': False, 'isConfigured': True}
-------------------
-------------------
[[[-56.505141902, 24.284177011], [-56.505141902, 72.663979307], [72.90613

In [69]:
## TESTING

top_left_x_list 	= []
top_left_y_list     = []
top_right_x_list    = []
top_right_y_list    = []
bottom_left_x_list  = []
bottom_left_y_list  = []
bottom_right_x_list = []
bottom_right_y_list = []


for tile in tiles:
    tile_extent=  tile['tileGeometry'] ['coordinates']
    

    top_left_x_list.append(tile_extent[0][0][0] )
    top_left_y_list.append(tile_extent[0][0][1]  )
    top_right_x_list.append(tile_extent[0][1][0]   )
    top_right_y_list.append(tile_extent[0][1][1]  )
    bottom_left_x_list.append( tile_extent[0][2][0]  )
    bottom_left_y_list.append( tile_extent[0][2][1]  )
    bottom_right_x_list.append(tile_extent[0][3][0]  )
    bottom_right_y_list.append(tile_extent[0][3][1]  )
    

    
    
print  (min(top_left_x_list))  ## left
print  (max(top_left_y_list))  ## top
print  (max(top_right_x_list))  ## right
print  (min(bottom_right_y_list))  ## bottom



900000.0
5500000.0
7400000.0
900000.0
